# Desafío PROPERATI
Equipo 4: Ester González, Ariel Groisman, Gastón Laterza, Facundo Oviedo, Gustavo Arias.


## Etapa 0:
### Importación de librerías y carga del dataset

1) Importamos ```Numpy```, ```Pandas``` y ```Seaborn```

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

Cargamos DataFrame de Properati y utilizamos su primer columna como INDEX

In [2]:
df = pd.read_csv('properatti.csv', index_col = 0)

## Etapa 1:
### Entender la información

 Vemos todas las columnas  y generamos una muestra de n=10

In [3]:
pd.set_option('display.max_columns', 500)
df.sample(10)

,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,lon,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
40059,sell,house,Santa Fe,|Argentina|Santa Fe|Santa Fe|,Argentina,Santa Fe,3836277.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,NaN,http://www.properati.com.ar/18qgd_venta_casa_s...,Corredor Responsable: Marcos Provvidenti - CCI...,CASA A TERMINAR EN VENTA AIRES DEL LLANO,https://thumbs4.properati.com/0/FnwEQwcdtvGl4I...
106772,sell,apartment,Rosario,|Argentina|Santa Fe|Rosario|,Argentina,Santa Fe,3838574.0,NaN,NaN,NaN,1242000.0,ARS,1229043.88,69655.92,NaN,35.0,NaN,35485.714286,NaN,NaN,NaN,http://www.properati.com.ar/1bwua_venta_depart...,"Excelente calidad, Monoambiente contrafrente.",Departamento - Macrocentro,NaN
103043,sell,apartment,Once,|Argentina|Capital Federal|Once|,Argentina,Capital Federal,3430291.0,NaN,NaN,NaN,112500.0,USD,1985006.25,112500.00,40.0,37.0,2812.500000,3040.540541,NaN,NaN,NaN,http://www.properati.com.ar/1brdc_venta_depart...,Excelente Edificio con posesión inmediata ubic...,BELLISIMO MONO-AMBIENTE DIVISIBLE.,https://thumbs4.properati.com/2/Orq0hU_GUE6AVL...
79309,sell,apartment,Santa Fe,|Argentina|Santa Fe|Santa Fe|,Argentina,Santa Fe,3836277.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,70.0,54.0,NaN,NaN,NaN,NaN,386.0,http://www.properati.com.ar/1avdv_venta_depart...,El departamento se encuentra en un edificio de...,Dos dormitorios con COCHERA - APTO CREDITO - e...,https://thumbs4.properati.com/6/i1h-YDISV6IT3i...
17382,sell,apartment,Nordelta,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre|Nord...,Argentina,Bs.As. G.B.A. Zona Norte,NaN,"-34.4265441653,-58.6570257238",-34.426544,-58.657026,114000.0,USD,2011473.00,114000.00,76.0,48.0,1500.000000,2375.000000,NaN,NaN,NaN,http://www.properati.com.ar/171p3_venta_depart...,CODIGO: 1695-722 ubicado en: HOMES - Publicad...,VENTA DEPARTAMENTO TIPO DÚPLEX 2 AMBIENTES EL...,https://thumbs4.properati.com/9/wtgyXauq5aLqvW...
4514,sell,apartment,Rosario,|Argentina|Santa Fe|Rosario|,Argentina,Santa Fe,3838574.0,"-32.9437056,-60.6743313",-32.943706,-60.674331,1546250.0,ARS,1530120.10,86719.38,62.0,39.0,1398.699677,39647.435897,NaN,NaN,NaN,http://www.properati.com.ar/15n36_venta_depart...,Rosario: San Juan 3698 (esquina Constitución) ...,Monoambiente divisible en Rosario: San Juan ...,https://thumbs4.properati.com/0/BZXb38Q3P--89G...
14057,sell,house,Barrio Cerrado Las Casuarinas,|Argentina|Bs.As. G.B.A. Zona Norte|Pilar|Barr...,Argentina,Bs.As. G.B.A. Zona Norte,NaN,"-34.3895496947,-58.8780105601",-34.389550,-58.878011,210000.0,USD,3705345.00,210000.00,220.0,180.0,954.545455,1166.666667,NaN,NaN,NaN,http://www.properati.com.ar/16r0m_venta_casa_p...,Espectacular propiedad desarrollada en dos pla...,Espectacular propiedad en Santa Guadalupe,https://thumbs4.properati.com/6/pN7o1KxsxZ97ho...
53019,sell,apartment,Villa Carlos Paz,|Argentina|Córdoba|Villa Carlos Paz|,Argentina,Córdoba,3832791.0,"-31.4015968,-64.5019937",-31.401597,-64.501994,100000.0,USD,1764450.00,100000.00,NaN,75.0,NaN,1333.333333,NaN,4.0,NaN,http://www.properati.com.ar/19kd8_venta_depart...,OPORTUNIDAD!! VILLA DEL LAGODPTO A LA CALLE EN...,DEPARTAMENTO VILLA DEL LAGO,https://thumbs4.properati.com/2/KQ2hUfZNZUpXaU...
96507,sell,apartment,Rosario,|Argentina|Santa Fe|Rosario|,Argentina,Santa Fe,3838574.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,63.0,63.0,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/1birl_venta_depart...,Sergio Villella Inmobiliaria presenta departam...,Paraguay 2100 - Departamento de un dormitorio ...,https://thumbs4.properati.com/6/XoRZSfTCiL1iqd...
52504,sell,PH,Monserrat,|Argentina|Capital Federal|Monserrat|,Argentina,Capital Federal,3430570.0,"-34.6143451,-58.3875796",-34.614345,-58.387580,160000.0,USD,2823120.00,160000.00,169.0,133.0,946.745562,1203.007519,NaN,1.0,NaN,http://www.properati.com.ar/19j8v_v

Una lectura inicial del datafame me permite observar la naturaleza del set de datos: publicaciones de propiedades para la venta, de distintas localidades de CABA, GBA y el interior.
Inicialmente:
- Disponemos de georeferencia en la mayoría de las observaciones. En algunos casos hay nulos en las columnas Latitud y Longitud, y en otras en el geopoint. Requiere limpieza.
- Podemos observar precios en ARS y USD. Este dato requiere mayor análisis y algún tipo de normalización.
- Hay campos de superficie de la propiedad que tienen muchos ```NaN```.
- El campo con las expensas presenta muchos nulos. Se podría pensar hay propiedades que no pagan expensas. Es necesario analizar este dato y adoptar una estrategia.
- El campo con cantidad de habitaciones tiene muchos nulos y, en principio, es muy importante para el análisis del dataset. Es posible extraer este dato de la descripción.

Acá quiero tirar unos boxplots

In [4]:
# BOXPLOT

Mediante el comando ```describe``` obtenemos información de estadísticos del dataset.

In [5]:
df.describe()

,geonames_id,lat,lon,price,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses
count,1.025030e+05,69670.000000,69670.000000,1.008100e+05,1.008100e+05,1.008100e+05,81892.000000,101313.000000,68617.000000,8.765800e+04,7899.000000,47390.000000,1.426200e+04
mean,3.574442e+06,-34.626210,-59.266290,4.685259e+05,4.229397e+06,2.397006e+05,233.795328,133.050181,2160.086916,6.912216e+03,17.452336,3.080840,5.009234e+03
std,3.541306e+05,1.980936,2.299922,2.260101e+06,6.904714e+06,3.913239e+05,1782.222147,724.351479,2759.288621,2.837864e+04,120.243621,1.860773,1.204403e+05
min,3.427208e+06,-54.823985,-75.678931,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.600000,1.510204e+00,1.000000,1.000000,1.000000e+00
25%,3.430234e+06,-34.669065,-58.727040,1.100000e+05,1.583309e+06,8.973388e+04,50.000000,45.000000,1218.181818,1.550000e+03,1.000000,2.000000,1.000000e+03
50%,3.433910e+06,-34.597985,-58.480128,1.850000e+05,2.558452e+06,1.450000e+05,84.000000,75.000000,1800.000000,2.213115e+03,3.000000,3.000000,2.000000e+03
75%,3.836668e+06,-34.441299,-58.395908,4.200000e+05,4.675792e+06,2.650000e+05,200.000000,150.000000,2486.411765,3.355549e+03,6.000000,4.000000,4.000000e+03
max,6.948895e+06,4.545843,-53.733330,6.500000e+08,8.212711e+08,4.654544e+07,200000.000000,187000.000000,206333.333333,4.000000e+06,3150.000000,32.000000,1.000150e+07


Uso ```shape``` para obtener la cantidad de Filas y Columnas


In [6]:
df.shape

(121220, 25)

Uso ```info``` para obtener los tipos de datos que tienen las columnas


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 121220 entries, 0 to 121219
Data columns (total 25 columns):
operation                     121220 non-null object
property_type                 121220 non-null object
place_name                    121197 non-null object
place_with_parent_names       121220 non-null object
country_name                  121220 non-null object
state_name                    121220 non-null object
geonames_id                   102503 non-null float64
lat-lon                       69670 non-null object
lat                           69670 non-null float64
lon                           69670 non-null float64
price                         100810 non-null float64
currency                      100809 non-null object
price_aprox_local_currency    100810 non-null float64
price_aprox_usd               100810 non-null float64
surface_total_in_m2           81892 non-null float64
surface_covered_in_m2         101313 non-null float64
price_usd_per_m2              68617 n

Puedo ver qué tipo de propiedades hay en el dataset, contando las observaciones desagregadas por el campo ```property_type```

In [8]:
df['property_type'].value_counts()

apartment    71065
house        40268
PH            5751
store         4136
Name: property_type, dtype: int64

La mayoría son departamentos, seguido por casas. 

In [9]:
df['property_type'].isnull().sum()

0

Ese campo no tiene nulos. Entre departamentos y casas, está representado el 90% de los datos.
Podemos continuar mirando la distribución geográfica

In [10]:
df['state_name'].value_counts()

Capital Federal                 32316
Bs.As. G.B.A. Zona Norte        25560
Bs.As. G.B.A. Zona Sur          13952
Córdoba                         12069
Santa Fe                        10172
Buenos Aires Costa Atlántica    10006
Bs.As. G.B.A. Zona Oeste         9322
Buenos Aires Interior            2291
Río Negro                         808
Neuquén                           733
Mendoza                           681
Tucumán                           674
Corrientes                        583
Misiones                          464
Entre Ríos                        369
Salta                             278
Chubut                            259
San Luis                          252
La Pampa                          157
Formosa                            65
Chaco                              57
San Juan                           40
Tierra Del Fuego                   31
Catamarca                          27
Jujuy                              26
Santa Cruz                         20
Santiago Del

Chequeamos si hay duplicados en el DataFrame


In [15]:
df[(df.duplicated() == True)]

,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,lon,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail


No hay ninguno. 


Analizamos el tipo de cambio PESOS/USD

In [27]:
tipo_de_cambio = (df['price_aprox_local_currency']/df['price_aprox_usd'])[1]
tipo_de_cambio

17.6445

Analizo la columna con monedas: ```currency```

In [28]:
df['currency'].value_counts()

USD    87587
ARS    13219
PEN        2
UYU        1
Name: currency, dtype: int64

Observamos tres registros de propiedades en el extranjero. Luego las vamos a eliminar

## Etapa 2:
### Limpieza de datos Inicial

Ya tenemos un panorama general del dataset. Ahora vamos a analizar puntualmente las necesidades de limpieza de datos.

Vamos a analizar la presencia de valores nulos en el dataset

In [11]:
df.isnull().sum()

operation                          0
property_type                      0
place_name                        23
place_with_parent_names            0
country_name                       0
state_name                         0
geonames_id                    18717
lat-lon                        51550
lat                            51550
lon                            51550
price                          20410
currency                       20411
price_aprox_local_currency     20410
price_aprox_usd                20410
surface_total_in_m2            39328
surface_covered_in_m2          19907
price_usd_per_m2               52603
price_per_m2                   33562
floor                         113321
rooms                          73830
expenses                      106958
properati_url                      0
description                        2
title                              0
image_thumbnail                 3112
dtype: int64

placename tiene solo 23 nulls, averiguamos cuales son:

In [18]:
df[(df.place_name.isnull() == True)].head()

,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,lon,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
6489,sell,house,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||,Argentina,Bs.As. G.B.A. Zona Norte,NaN,"-34.4008968545,-58.63809847",-34.400897,-58.638098,650000.0,USD,11468925.0,650000.0,240.0,240.0,2708.333333,2708.333333,NaN,NaN,NaN,http://www.properati.com.ar/15ui7_venta_casa_t...,"Espectacular casa moderna de una planta, con a...",Venta de casa en Santa María de Tigre,https://thumbs4.properati.com/7/vQjgbHSz1fKXxy...
10201,sell,house,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||,Argentina,Bs.As. G.B.A. Zona Norte,NaN,"-34.400704,-58.638067",-34.400704,-58.638067,535000.0,USD,9439807.5,535000.0,NaN,300.0,NaN,1783.333333,NaN,5.0,NaN,http://www.properati.com.ar/166p5_venta_casa_t...,Corredor Responsable: SABRINA COCCONI - CSI 62...,VENTA CASA SANTA MARÍA DE TIGRE COUNTRY BARRIO...,https://thumbs4.properati.com/1/FABpKIJMaoAO3t...
11451,sell,house,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||,Argentina,Bs.As. G.B.A. Zona Norte,NaN,"-34.4026880384,-58.6262613379",-34.402688,-58.626261,550000.0,USD,9704475.0,550000.0,2170.0,270.0,253.456221,2037.037037,NaN,NaN,NaN,http://www.properati.com.ar/16bpd_venta_casa_t...,MUY LINDA CASA EN EXCELENTE UBICACION!! Superf...,SANTA MARIA DE TIGRE IMPECABLE,https://thumbs4.properati.com/5/fwH9L71hW2rW7V...
14839,sell,house,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||,Argentina,Bs.As. G.B.A. Zona Norte,NaN,"-34.4007994779,-58.6381735719",-34.400799,-58.638174,480000.0,USD,8469360.0,480000.0,240.0,200.0,2000.000000,2400.000000,NaN,NaN,NaN,http://www.properati.com.ar/16tal_venta_casa_b...,Muy linda casa de estilo clásico.-Planta baja;...,Venta de casa en Santa María de Tigre,https://thumbs4.properati.com/1/3ZbPSffTY38y7v...
18622,sell,house,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||,Argentina,Bs.As. G.B.A. Zona Norte,NaN,"-34.4032781195,-58.6263503945",-34.403278,-58.626350,1297000.0,USD,22884916.5,1297000.0,340.0,340.0,3814.705882,3814.705882,NaN,NaN,NaN,http://www.properati.com.ar/177ee_venta_casa_t...,Casa con increible vista al lago con playa de...,Casa con increible vista al lago con playa y m...,https://thumbs4.properati.com/0/52JCH1PKYCOMe6...


Ya que todos pertenecen a Tigre, podemos reemplazarlos fácilmente:

In [ ]:
df['place_name'] = df['place_name'].fillna("Tigre")

Continuamos eliminando todas las columnas no relevantes para el presente trabajo.

In [12]:
#df = df.drop (columns = ["lat-lon", "image_thumbnail", "operation", "properati_url", "country_name", "price","currency"])


Eliminamos los registros con información de Perú y Uruguay


In [ ]:
#df[df.currency != 'PER']
#df[df.currency != 'URU']
#df.head(20)

Detectamos que hay unos registros con Place Name “Centro” que corresponden todos a Mar del Plata


In [14]:
df[df['place_name'] == 'Centro'].head(231)


,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,lon,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
4,sell,apartment,Centro,|Argentina|Buenos Aires Costa Atlántica|Mar de...,Argentina,Buenos Aires Costa Atlántica,3435548.0,"-38.0026256,-57.5494468",-38.002626,-57.549447,64000.0,USD,1129248.00,64000.00,35.0,35.0,1828.571429,1828.571429,NaN,NaN,NaN,http://www.properati.com.ar/15bok_venta_depart...,DEPARTAMENTO CON FANTÁSTICA ILUMINACIÓN NATURA...,DEPTO 2 AMB AL CONTRAFRENTE ZONA CENTRO/PLAZA ...,https://thumbs4.properati.com/5/xrRqlNcSI_vs-f...
5187,sell,apartment,Centro,|Argentina|Buenos Aires Costa Atlántica|Mar de...,Argentina,Buenos Aires Costa Atlántica,3435548.0,"-38.0046431,-57.5454812",-38.004643,-57.545481,59000.0,USD,1041025.50,59000.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15pez_venta_depart...,VENTA DEPARTAMENTO DE 2 AMBIENTES LATERAL MUY ...,VENTA DEPARTAMENTO ZONA CENTRO 2 AMBIENTES LA...,https://thumbs4.properati.com/6/cW95nF2RX3-J2t...
6109,sell,apartment,Centro,|Argentina|Buenos Aires Costa Atlántica|Mar de...,Argentina,Buenos Aires Costa Atlántica,3435548.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,83.0,NaN,NaN,NaN,10.0,4.0,NaN,http://www.properati.com.ar/15ssk_venta_depart...,"Dpto en P.H Edificio Plaza. Piso 10 dpto E , ...",DEPARTAMENTO EN VENTA,https://thumbs4.properati.com/6/btJ9fVU_pAFJpX...
8644,sell,store,Centro,|Argentina|Buenos Aires Costa Atlántica|Mar de...,Argentina,Buenos Aires Costa Atlántica,3435548.0,"-38.0031696,-57.5485722",-38.003170,-57.548572,120000.0,USD,2117340.00,120000.00,30.0,30.0,4000.000000,4000.000000,NaN,NaN,NaN,http://www.properati.com.ar/161p5_venta_local_...,LOCAL EN VENTA CENTRO. Oportunidad buena obica...,VENTA DE LOCAL,https://thumbs4.properati.com/5/ciQ6xMZ4-oyTF_...
10349,sell,apartment,Centro,|Argentina|Buenos Aires Costa Atlántica|Mar de...,Argentina,Buenos Aires Costa Atlántica,3435548.0,"-37.998698,-57.546018",-37.998698,-57.546018,52000.0,USD,917514.00,52000.00,33.0,NaN,1575.757576,NaN,NaN,2.0,NaN,http://www.properati.com.ar/1679a_venta_depart...,Departamento de 2 ambientes contrafrente en Ex...,Departamento de 2 ambientes en el centro. Sobr...,https://thumbs4.properati.com/1/SpV1V2VPYN9w2a...
12069,sell,apartment,Centro,|Argentina|Buenos Aires Costa Atlántica|Mar de...,Argentina,Buenos Aires Costa Atlántica,3435548.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,240.0,200.0,NaN,NaN,NaN,4.0,NaN,http://www.properati.com.ar/16j4k_venta_depart...,VENTA DEPARTAMENTO 4 AMBIENTES EN MAR DEL PLAT...,DEPARTAMENTO EN VENTA,https://thumbs4.properati.com/9/BwKm9PFVZOJFq3...
12076,sell,store,Centro,|Argentina|Buenos Aires Costa Atlántica|Mar de...,Argentina,Buenos Aires Costa Atlántica,3435548.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,70.0,70.0,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/16j4z_venta_local_...,LOCALVENTA LOCAL EN MAR DEL PLATA.LOCAL DE 3 x...,LOCAL EN VENTA,https://thumbs4.properati.com/8/dV0kbMw6GOknzS...
12123,sell,apartment,Centro,|Argentina|Buenos Aires Costa Atlántica|Mar de...,Argentina,Buenos Aires Costa Atlántica,3435548.0,NaN,NaN,NaN,65000.0,USD,1146892.50,65000.00,46.0,46.0,1413.043478,1413.043478,NaN,2.0,400.0,http://www.properati.com.ar/16j7n_venta_depart...,VENTA DEPARTAMENTO 2 AMBIENTES EN MAR DEL PLAT...,DEPARTAMENTO EN VENTA,https://thumbs4.properati.com/4/UwRO95HvHC3Bb5...
12124,sell,apartment,Centro,|Argentina|Buenos Aires Costa Atlántica|Mar de...,Argentina,Buenos Aires Costa Atlántica,3435548.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,56.0,56.0,NaN,NaN,NaN,2.0,1800.0,http://www.properati.com.ar/16j7u_venta_depart...,VENTA DE DEPARTAMENTO EN MAR DEL PLATADEPARTAM...,DEPARTAMENTO EN VENTA,https://thumbs4.properati.com/5/hMp8VjAg8ue1bI...
12126,sell,apartment,Centro,|Argentina|Buenos Aires Costa Atlántica|Mar de...,Argentina,Buenos Aires Costa Atlántica,3435548.0,NaN,NaN,NaN

Los transformamos “manualmente”

In [ ]:
df = df.replace('Centro', 'Mar del Plata')
df.head(30)

Verifico

In [ ]:
df[df['place_name'] == 'Centro'].head(231)
# No quedo ninguno

Veo que no haya valores en moneda local y que falte en dólares, dado que me quiero quedar sólo con la columna en dólares.

In [ ]:
df[(df.price_aprox_local_currency.isnull() == False) & (df.price_aprox_usd.isnull() == True)]

### Problema 1: Número de ambientes

Observamos en el análisis que el campo ```rooms``` , número de ambientes, trae información nula. Este campo será muy importante al momento de estimar los drivers de los precios de las propiedades.
También observamos que esta información generalmente está contenida en la descripción de la publicación. Mediante la función ```REGEX```, vamos a recupera el número de ambientes y poblar el feature correspondiente.

Creamos un procedimiento con ```REGEX``` para buscar numero de ambientes:

In [20]:
def num_ambientes(column):
    return re.findall('([0-9]+)\s?amb[a-z]*',column,flags = re.IGNORECASE)

Pasamos la columna description a string para poder aplicar la función

In [21]:
df['description'] = df['description'].astype(str)

La aplicamos en la columna description y arroja el resultado sobre una nueva columna

In [22]:
df['Num_Ambientes'] = df['description'].apply(num_ambientes)

NameError: name 're' is not defined

Funciona, pero solo sobre el 25% del DataFrame, hay descripciones muy problematicas, ejemplo:

In [24]:
df.loc[24614].description

'"COMPLEJO ERRE FULL AMENITIES" Se trata de un conjunto formado por 3 edificios con planta baja común, local comercial, 9 pisos altos sobre Av. Monroe, 4 pisos altos sobre calle Freire y 4 pisos altos sobre calle Blanco Encalada. 2 subsuelos para estacionamiento propio, locales de servicios generales, hall de acceso peatonal único, jardín con piscina en planta baja y terraza jardín con amenities en terraza sobre Av. Monroe. El conjunto se organiza mediante tres núcleos de circulaciones verticales independientes que conectan las unidades de uso flexible con el lobby de acceso, cocheras y demás áreas comunes.Características generales del conjuntoPlanta Baja Hall de acceso peatonal único sobre la calle Freire y vehicular sobre las calles Freire y Blanco Encalada. Lobby de entrada a cada conjunto con frente vidriado, puerta de acceso, piso de mármol, granito natural o porcelanato y revestimientos de acero inoxidable, espejo, mármol o madera. Accesos a estacionamientos.Regulador y medidor d

Mirar lo que pasa si hacemos un value counts sobre la nueva columna creada para tener una idea:

In [25]:
df['Num_Ambientes'].value_counts()

KeyError: 'Num_Ambientes'

### Problema 2: Llenar la columna precios en dólares ```price```


### Problema 3: Tomar una decisión con las columnas con precios por metro cuadrado

### Problema 4: Consolidar la información geográfica
